# Batch process segmentation made with Ilastik

This code performs the identical steps as in notebook 3, but in a more streamlined way, allowing for batch processing without visual inspection.

The processing functions have been moved to the process_colonies.py script.

In [8]:
#main data analysis packages
import numpy as np
import pandas as pd
import dask.array as da

#next line is required for Napari
%gui qt

#image viewer
import napari

#path handling
import pathlib
import h5py

#dask cash
from dask.cache import Cache
cache = Cache(4e9)  # Leverage 4 gigabytes of memory
cache.register()    # Turn cache on globally

import process_colonies as pc

## Set Paths and Settings

In [5]:
#set path to registered file
path_reg_im = pathlib.Path("/Users/simonvanvliet/TempData")

#set path to Ilastik output file
path_seg_im = pathlib.Path("/Users/simonvanvliet/TempData")

#set path to temporary store label images (use local machine folder for speed)
temp_data_path = pathlib.Path.home() / 'TempData'
temp_data_path.mkdir(exist_ok=True)

#set path to output csv files
path_data_files = pathlib.Path('/Users/simonvanvliet/TempData/DataFiles/') 
path_data_files.mkdir(exist_ok=True)

#set filenames
exp_name_list = ["20230411","20230427","20230504"]

#specify properties to extract 
prop_list = ['label', 
            'area', 'centroid', 
            'axis_major_length', 'axis_minor_length']

#specify processing settings
settings = {
    'calc_edge_dist'    : True, #set to true to calculate distance between colony edges, more accurate that center to center distance, but very slow
    'prop_list'         : prop_list,
    #specify the order of the strains in the Ilastik layers
    #specify the order of the strains in the Ilastik layers
    'idx_SA1'   : 1, #SA1 is GFP
    'idx_SA2'   : 0, #SA2 is RFP
    'idx_BG'    : 2,
    'idx_PA'    : 3,
    #specify the segementation processing parameters for pseudomonas
    'sigma'             : 1, # sigma for gaussian filter
    'threshold_PA'      : 0.5, # threshold for segmentation
    'closing_radius_PA' : 5, # radius for closing operation
    'min_cell_area_PA'  : 50, # minimum area for a cell to be considered
    'max_hole_area_PA'  : 5000, # maximum area for a hole to be filled
    #specify the segementation processing parameters for staph
    'sigma'             : 2, # sigma for gaussian filter
    'threshold_SA'      : 0.5, # threshold for segmentation
    'closing_radius_SA' : 5, # radius for closing operation
    'min_cell_area_SA'  : 50, # minimum area for a cell to be considered
    'max_hole_area_SA'  : 1000, # maximum area for a hole to be filled    
    #store path metadata
    'temp_path'         : temp_data_path,
    'path_seg_im_root'  : path_seg_im,
    'path_reg_im_root'  : path_reg_im,
    'path_data_files'    : path_data_files,
    'exp_name_list'     : exp_name_list
}

## Loop experiments and positions
First make sure that all positions are found

In [11]:
path_list = [] 
for exp_name in exp_name_list:
    seg_path = settings['path_reg_im_root'] / f'{exp_name}-agar-pad-processed' 
    pos_list = [f.name for f in sorted(seg_path.glob('*[0-9].h5'))]
    
    for pos in pos_list:
        path_list.append(seg_path / pos)
print(len(path_list))

72


now we loop positions, this will take a while

In [31]:
phase_all = None
for path in path_list:
    #load segmented images
    im_file = h5py.File(path, 'r') #open 
    chunk_size = (1, 1,*im_file['images'].shape[-2:])
    images = da.from_array(im_file['images'], chunks=chunk_size)
    phase = images[:19,:,:,:][np.newaxis,...] 
    phase_all = da.concatenate((phase_all, phase), axis=0) if phase_all is not None else phase

In [33]:
viewer = napari.view_image(phase_all,
            channel_axis=2,
            name=["SA1-RFP", "SA2-GFP", "PA-Phase"],
            colormap=["red", "green", "gray"])
napari.run()
napari.run()